In [1]:
import pandas as pd
import numpy as np
import random
import randomcolor

In [2]:
import plotly.express as px
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [3]:
def gen_colors(count:int):
    result = set()
    while len(result) != count:
        result.update(randomcolor.RandomColor().generate())
    
    return list(result)

In [4]:
def check_1by1(df: pd.DataFrame, group1: list, group2: list):
    cb_cols = group1 + group2
    
    cb_vc = len(df.groupby(by=cb_cols))
    g1_vc = df[group1].nunique().iloc[0]
    g2_vc = df[group2].nunique().iloc[0]
    
    return g1_vc==g2_vc==cb_vc

# Load Data

In [5]:
MILESTONES_PATH = 'IBI_MILESTONES_TRAIN.csv'
MILESTONES = pd.read_csv(MILESTONES_PATH)

COMMENTS_PATH = 'IBI_COMMENTS_TRAIN.csv'
COMMENTS = pd.read_csv(COMMENTS_PATH)

METADATA_PATH = 'IBI_METADATA_TRAIN.csv'
METADATA = pd.read_csv(METADATA_PATH)

DICTIONARY_PATH = 'challenge_dictionary_info.csv'
DICTIONARY = pd.read_csv(DICTIONARY_PATH)
DICTIONARY = DICTIONARY.sort_values(by=['count'], ascending=False)

c:\users\puff\anaconda3\envs\ieee\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


# VIEW DATA - NOTEDESCRIPTION

In [6]:
temp = MILESTONES[~MILESTONES['NOTEDESCRIPTION'].isin(['6763', 'uJuK'])]
note_cat = temp['NOTEDESCRIPTION'].value_counts().rename_axis('value_cat').reset_index(name='counts')
note_cat = note_cat[note_cat['counts']>1]

In [7]:
def get_pos(val, df):
    val = val.strip()
    tokens = val.split(' ')
    if len(tokens) == 1:
        word = tokens[0]
        pos = df[df['id']==word]['pos'].values[0]
        return pos
    else:
        return 'MULTI'


note_cat['pos'] = note_cat.apply(lambda x: get_pos(x['value_cat'], DICTIONARY), axis=1)

In [8]:
note_cat_chunk = note_cat.head(30)

In [9]:
def get_refs(val, df):
    temp = df[df['NOTEDESCRIPTION']==val]['REFERENCEID'].unique()
    return len(temp)


note_cat_chunk['refs'] = note_cat_chunk.apply(lambda x: get_refs(x['value_cat'], MILESTONES), axis=1)

c:\users\puff\anaconda3\envs\ieee\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [10]:
note_cat_chunk['ref_rate'] = (note_cat_chunk['refs'] / len(METADATA))

c:\users\puff\anaconda3\envs\ieee\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


## Milestone Note appearance

In [30]:
fig = px.bar(
    note_cat_chunk, 
    x='value_cat', y='counts', 
    color='pos',
    title='Milestone Note appearance'
)
fig.update_layout(xaxis_categoryorder = 'total descending')
fig.show()

In [12]:
note_cat_chunk_ref = note_cat_chunk.sort_values(by=['ref_rate']).reset_index(drop=True)

## Milestone Note with Referece Freqency

In [15]:
fig = px.bar(
    note_cat_chunk_ref, 
    x='value_cat', y='ref_rate', 
    color='pos',
    title='Milestone Note with Referece Freqency'
)
fig.update_layout(xaxis_categoryorder = 'total descending')
fig.show()

# VIEW DATA - MILESTONEID
# MILESTONEID -> MILESTONETYPE

In [16]:
check_1by1(MILESTONES, ['MILESTONEID'], ['MILESTONEDESCRIPTION'])

False

In [17]:
ms_cat = MILESTONES['MILESTONEID'].value_counts().rename_axis('value_cat').reset_index(name='counts')
ms_cat = ms_cat.sort_values('value_cat').reset_index(drop=True)
len(ms_cat)

67

## MILESTONEID Frequency

In [18]:
fig = px.bar(
    ms_cat, 
    x='value_cat', y='counts',
    color='value_cat',
    title='MILESTONEID Frequency'
)
fig.show()

# Milestones Trend with Reference

In [22]:
ref_id_list = list(MILESTONES['REFERENCEID'].unique())
picked = random.sample(ref_id_list, k=100)
ms_chunk = MILESTONES[MILESTONES['REFERENCEID'].isin(picked)]

In [23]:
fig = px.line(ms_chunk, 
              x='SECONDS_SINCE_CASE_START', y='MILESTONEID', 
              color='REFERENCEID',
              line_group='REFERENCEID', 
              color_discrete_sequence=gen_colors(100),
              title='Milestones Trend with Reference'
             )
fig.show()

# MILESTONE - CREATED_BY & COMMENTS - GLOBALID

In [24]:
milestone = MILESTONES[~MILESTONES['CREATED_BY'].isin(['6763', 'uJuK'])]
comments = COMMENTS[~COMMENTS['GLOBALID'].isin(['6763', 'uJuK'])]

In [25]:
milestone_gb_ID = milestone.groupby(['REFERENCEID'])

In [26]:
mile_and_com = pd.DataFrame(columns=['REFERENCEID', 'is_corr'])
for index, group in enumerate(milestone_gb_ID):
    print('{} / {}'.format(index+1, len(milestone_gb_ID)))
    group_name = group[0]
    
    df = group[1]
    mile_create = set(df['CREATED_BY'].unique())
    
    com_df = comments[comments['REFERENCEID']==group_name]
    com_glob = set(com_df['GLOBALID'].unique())
    
    same = mile_create.intersection(com_glob)
    short = min(len(mile_create), len(com_glob))
    
    if len(same) >= short:
        check = True
    else:
        check = False
    
    mile_and_com.loc[len(mile_and_com)] = [group_name, check]

1 / 40911
2 / 40911
3 / 40911
4 / 40911
5 / 40911
6 / 40911
7 / 40911
8 / 40911
9 / 40911
10 / 40911
11 / 40911
12 / 40911
13 / 40911
14 / 40911
15 / 40911
16 / 40911
17 / 40911
18 / 40911
19 / 40911
20 / 40911
21 / 40911
22 / 40911
23 / 40911
24 / 40911
25 / 40911
26 / 40911
27 / 40911
28 / 40911
29 / 40911
30 / 40911
31 / 40911
32 / 40911
33 / 40911
34 / 40911
35 / 40911
36 / 40911
37 / 40911
38 / 40911
39 / 40911
40 / 40911
41 / 40911
42 / 40911
43 / 40911
44 / 40911
45 / 40911
46 / 40911
47 / 40911
48 / 40911
49 / 40911
50 / 40911
51 / 40911
52 / 40911
53 / 40911
54 / 40911
55 / 40911
56 / 40911
57 / 40911
58 / 40911
59 / 40911
60 / 40911
61 / 40911
62 / 40911
63 / 40911
64 / 40911
65 / 40911
66 / 40911
67 / 40911
68 / 40911
69 / 40911
70 / 40911
71 / 40911
72 / 40911
73 / 40911
74 / 40911
75 / 40911
76 / 40911
77 / 40911
78 / 40911
79 / 40911
80 / 40911
81 / 40911
82 / 40911
83 / 40911
84 / 40911
85 / 40911
86 / 40911
87 / 40911
88 / 40911
89 / 40911
90 / 40911
91 / 40911
92 / 409

694 / 40911
695 / 40911
696 / 40911
697 / 40911
698 / 40911
699 / 40911
700 / 40911
701 / 40911
702 / 40911
703 / 40911
704 / 40911
705 / 40911
706 / 40911
707 / 40911
708 / 40911
709 / 40911
710 / 40911
711 / 40911
712 / 40911
713 / 40911
714 / 40911
715 / 40911
716 / 40911
717 / 40911
718 / 40911
719 / 40911
720 / 40911
721 / 40911
722 / 40911
723 / 40911
724 / 40911
725 / 40911
726 / 40911
727 / 40911
728 / 40911
729 / 40911
730 / 40911
731 / 40911
732 / 40911
733 / 40911
734 / 40911
735 / 40911
736 / 40911
737 / 40911
738 / 40911
739 / 40911
740 / 40911
741 / 40911
742 / 40911
743 / 40911
744 / 40911
745 / 40911
746 / 40911
747 / 40911
748 / 40911
749 / 40911
750 / 40911
751 / 40911
752 / 40911
753 / 40911
754 / 40911
755 / 40911
756 / 40911
757 / 40911
758 / 40911
759 / 40911
760 / 40911
761 / 40911
762 / 40911
763 / 40911
764 / 40911
765 / 40911
766 / 40911
767 / 40911
768 / 40911
769 / 40911
770 / 40911
771 / 40911
772 / 40911
773 / 40911
774 / 40911
775 / 40911
776 / 40911
777 

1356 / 40911
1357 / 40911
1358 / 40911
1359 / 40911
1360 / 40911
1361 / 40911
1362 / 40911
1363 / 40911
1364 / 40911
1365 / 40911
1366 / 40911
1367 / 40911
1368 / 40911
1369 / 40911
1370 / 40911
1371 / 40911
1372 / 40911
1373 / 40911
1374 / 40911
1375 / 40911
1376 / 40911
1377 / 40911
1378 / 40911
1379 / 40911
1380 / 40911
1381 / 40911
1382 / 40911
1383 / 40911
1384 / 40911
1385 / 40911
1386 / 40911
1387 / 40911
1388 / 40911
1389 / 40911
1390 / 40911
1391 / 40911
1392 / 40911
1393 / 40911
1394 / 40911
1395 / 40911
1396 / 40911
1397 / 40911
1398 / 40911
1399 / 40911
1400 / 40911
1401 / 40911
1402 / 40911
1403 / 40911
1404 / 40911
1405 / 40911
1406 / 40911
1407 / 40911
1408 / 40911
1409 / 40911
1410 / 40911
1411 / 40911
1412 / 40911
1413 / 40911
1414 / 40911
1415 / 40911
1416 / 40911
1417 / 40911
1418 / 40911
1419 / 40911
1420 / 40911
1421 / 40911
1422 / 40911
1423 / 40911
1424 / 40911
1425 / 40911
1426 / 40911
1427 / 40911
1428 / 40911
1429 / 40911
1430 / 40911
1431 / 40911
1432 / 40911

2026 / 40911
2027 / 40911
2028 / 40911
2029 / 40911
2030 / 40911
2031 / 40911
2032 / 40911
2033 / 40911
2034 / 40911
2035 / 40911
2036 / 40911
2037 / 40911
2038 / 40911
2039 / 40911
2040 / 40911
2041 / 40911
2042 / 40911
2043 / 40911
2044 / 40911
2045 / 40911
2046 / 40911
2047 / 40911
2048 / 40911
2049 / 40911
2050 / 40911
2051 / 40911
2052 / 40911
2053 / 40911
2054 / 40911
2055 / 40911
2056 / 40911
2057 / 40911
2058 / 40911
2059 / 40911
2060 / 40911
2061 / 40911
2062 / 40911
2063 / 40911
2064 / 40911
2065 / 40911
2066 / 40911
2067 / 40911
2068 / 40911
2069 / 40911
2070 / 40911
2071 / 40911
2072 / 40911
2073 / 40911
2074 / 40911
2075 / 40911
2076 / 40911
2077 / 40911
2078 / 40911
2079 / 40911
2080 / 40911
2081 / 40911
2082 / 40911
2083 / 40911
2084 / 40911
2085 / 40911
2086 / 40911
2087 / 40911
2088 / 40911
2089 / 40911
2090 / 40911
2091 / 40911
2092 / 40911
2093 / 40911
2094 / 40911
2095 / 40911
2096 / 40911
2097 / 40911
2098 / 40911
2099 / 40911
2100 / 40911
2101 / 40911
2102 / 40911

2666 / 40911
2667 / 40911
2668 / 40911
2669 / 40911
2670 / 40911
2671 / 40911
2672 / 40911
2673 / 40911
2674 / 40911
2675 / 40911
2676 / 40911
2677 / 40911
2678 / 40911
2679 / 40911
2680 / 40911
2681 / 40911
2682 / 40911
2683 / 40911
2684 / 40911
2685 / 40911
2686 / 40911
2687 / 40911
2688 / 40911
2689 / 40911
2690 / 40911
2691 / 40911
2692 / 40911
2693 / 40911
2694 / 40911
2695 / 40911
2696 / 40911
2697 / 40911
2698 / 40911
2699 / 40911
2700 / 40911
2701 / 40911
2702 / 40911
2703 / 40911
2704 / 40911
2705 / 40911
2706 / 40911
2707 / 40911
2708 / 40911
2709 / 40911
2710 / 40911
2711 / 40911
2712 / 40911
2713 / 40911
2714 / 40911
2715 / 40911
2716 / 40911
2717 / 40911
2718 / 40911
2719 / 40911
2720 / 40911
2721 / 40911
2722 / 40911
2723 / 40911
2724 / 40911
2725 / 40911
2726 / 40911
2727 / 40911
2728 / 40911
2729 / 40911
2730 / 40911
2731 / 40911
2732 / 40911
2733 / 40911
2734 / 40911
2735 / 40911
2736 / 40911
2737 / 40911
2738 / 40911
2739 / 40911
2740 / 40911
2741 / 40911
2742 / 40911

...

38090 / 40911
38091 / 40911
38092 / 40911
38093 / 40911
38094 / 40911
38095 / 40911
38096 / 40911
38097 / 40911
38098 / 40911
38099 / 40911
38100 / 40911
38101 / 40911
38102 / 40911
38103 / 40911
38104 / 40911
38105 / 40911
38106 / 40911
38107 / 40911
38108 / 40911
38109 / 40911
38110 / 40911
38111 / 40911
38112 / 40911
38113 / 40911
38114 / 40911
38115 / 40911
38116 / 40911
38117 / 40911
38118 / 40911
38119 / 40911
38120 / 40911
38121 / 40911
38122 / 40911
38123 / 40911
38124 / 40911
38125 / 40911
38126 / 40911
38127 / 40911
38128 / 40911
38129 / 40911
38130 / 40911
38131 / 40911
38132 / 40911
38133 / 40911
38134 / 40911
38135 / 40911
38136 / 40911
38137 / 40911
38138 / 40911
38139 / 40911
38140 / 40911
38141 / 40911
38142 / 40911
38143 / 40911
38144 / 40911
38145 / 40911
38146 / 40911
38147 / 40911
38148 / 40911
38149 / 40911
38150 / 40911
38151 / 40911
38152 / 40911
38153 / 40911
38154 / 40911
38155 / 40911
38156 / 40911
38157 / 40911
38158 / 40911
38159 / 40911
38160 / 40911
38161 

38699 / 40911
38700 / 40911
38701 / 40911
38702 / 40911
38703 / 40911
38704 / 40911
38705 / 40911
38706 / 40911
38707 / 40911
38708 / 40911
38709 / 40911
38710 / 40911
38711 / 40911
38712 / 40911
38713 / 40911
38714 / 40911
38715 / 40911
38716 / 40911
38717 / 40911
38718 / 40911
38719 / 40911
38720 / 40911
38721 / 40911
38722 / 40911
38723 / 40911
38724 / 40911
38725 / 40911
38726 / 40911
38727 / 40911
38728 / 40911
38729 / 40911
38730 / 40911
38731 / 40911
38732 / 40911
38733 / 40911
38734 / 40911
38735 / 40911
38736 / 40911
38737 / 40911
38738 / 40911
38739 / 40911
38740 / 40911
38741 / 40911
38742 / 40911
38743 / 40911
38744 / 40911
38745 / 40911
38746 / 40911
38747 / 40911
38748 / 40911
38749 / 40911
38750 / 40911
38751 / 40911
38752 / 40911
38753 / 40911
38754 / 40911
38755 / 40911
38756 / 40911
38757 / 40911
38758 / 40911
38759 / 40911
38760 / 40911
38761 / 40911
38762 / 40911
38763 / 40911
38764 / 40911
38765 / 40911
38766 / 40911
38767 / 40911
38768 / 40911
38769 / 40911
38770 

39309 / 40911
39310 / 40911
39311 / 40911
39312 / 40911
39313 / 40911
39314 / 40911
39315 / 40911
39316 / 40911
39317 / 40911
39318 / 40911
39319 / 40911
39320 / 40911
39321 / 40911
39322 / 40911
39323 / 40911
39324 / 40911
39325 / 40911
39326 / 40911
39327 / 40911
39328 / 40911
39329 / 40911
39330 / 40911
39331 / 40911
39332 / 40911
39333 / 40911
39334 / 40911
39335 / 40911
39336 / 40911
39337 / 40911
39338 / 40911
39339 / 40911
39340 / 40911
39341 / 40911
39342 / 40911
39343 / 40911
39344 / 40911
39345 / 40911
39346 / 40911
39347 / 40911
39348 / 40911
39349 / 40911
39350 / 40911
39351 / 40911
39352 / 40911
39353 / 40911
39354 / 40911
39355 / 40911
39356 / 40911
39357 / 40911
39358 / 40911
39359 / 40911
39360 / 40911
39361 / 40911
39362 / 40911
39363 / 40911
39364 / 40911
39365 / 40911
39366 / 40911
39367 / 40911
39368 / 40911
39369 / 40911
39370 / 40911
39371 / 40911
39372 / 40911
39373 / 40911
39374 / 40911
39375 / 40911
39376 / 40911
39377 / 40911
39378 / 40911
39379 / 40911
39380 

39909 / 40911
39910 / 40911
39911 / 40911
39912 / 40911
39913 / 40911
39914 / 40911
39915 / 40911
39916 / 40911
39917 / 40911
39918 / 40911
39919 / 40911
39920 / 40911
39921 / 40911
39922 / 40911
39923 / 40911
39924 / 40911
39925 / 40911
39926 / 40911
39927 / 40911
39928 / 40911
39929 / 40911
39930 / 40911
39931 / 40911
39932 / 40911
39933 / 40911
39934 / 40911
39935 / 40911
39936 / 40911
39937 / 40911
39938 / 40911
39939 / 40911
39940 / 40911
39941 / 40911
39942 / 40911
39943 / 40911
39944 / 40911
39945 / 40911
39946 / 40911
39947 / 40911
39948 / 40911
39949 / 40911
39950 / 40911
39951 / 40911
39952 / 40911
39953 / 40911
39954 / 40911
39955 / 40911
39956 / 40911
39957 / 40911
39958 / 40911
39959 / 40911
39960 / 40911
39961 / 40911
39962 / 40911
39963 / 40911
39964 / 40911
39965 / 40911
39966 / 40911
39967 / 40911
39968 / 40911
39969 / 40911
39970 / 40911
39971 / 40911
39972 / 40911
39973 / 40911
39974 / 40911
39975 / 40911
39976 / 40911
39977 / 40911
39978 / 40911
39979 / 40911
39980 

40509 / 40911
40510 / 40911
40511 / 40911
40512 / 40911
40513 / 40911
40514 / 40911
40515 / 40911
40516 / 40911
40517 / 40911
40518 / 40911
40519 / 40911
40520 / 40911
40521 / 40911
40522 / 40911
40523 / 40911
40524 / 40911
40525 / 40911
40526 / 40911
40527 / 40911
40528 / 40911
40529 / 40911
40530 / 40911
40531 / 40911
40532 / 40911
40533 / 40911
40534 / 40911
40535 / 40911
40536 / 40911
40537 / 40911
40538 / 40911
40539 / 40911
40540 / 40911
40541 / 40911
40542 / 40911
40543 / 40911
40544 / 40911
40545 / 40911
40546 / 40911
40547 / 40911
40548 / 40911
40549 / 40911
40550 / 40911
40551 / 40911
40552 / 40911
40553 / 40911
40554 / 40911
40555 / 40911
40556 / 40911
40557 / 40911
40558 / 40911
40559 / 40911
40560 / 40911
40561 / 40911
40562 / 40911
40563 / 40911
40564 / 40911
40565 / 40911
40566 / 40911
40567 / 40911
40568 / 40911
40569 / 40911
40570 / 40911
40571 / 40911
40572 / 40911
40573 / 40911
40574 / 40911
40575 / 40911
40576 / 40911
40577 / 40911
40578 / 40911
40579 / 40911
40580 

In [27]:
mile_and_com_corr = mile_and_com['is_corr'].value_counts().rename_axis('value_cat').reset_index(name='counts')

In [28]:
mile_and_com_corr

,value_cat,counts
0,True,38622
1,False,2289


In [29]:
fig = px.pie(
    mile_and_com_corr, 
    values='counts', names='value_cat',
    title='Correlation between MILESTONE - CREATED_BY & COMMENTS - GLOBALID'
            )
fig.show()